In [86]:
from pyscf import gto, scf
import numpy as np
import scipy as sp
import sys
sys.path.append('/workspaces/Huckel-on-benzene/project/Benzene/rhf')
import plottingsystem as pts

In [87]:
benzene_molecule = '''
  C    1.2116068    0.6995215    0.0000000
  C    1.2116068   -0.6995215    0.0000000
  C   -0.0000000   -1.3990430   -0.0000000
  C   -1.2116068   -0.6995215   -0.0000000
  C   -1.2116068    0.6995215   -0.0000000
  C    0.0000000    1.3990430    0.0000000
  H    2.1489398    1.2406910    0.0000000
  H    2.1489398   -1.2406910    0.0000000
  H   -0.0000000   -2.4813820   -0.0000000
  H   -2.1489398   -1.2406910   -0.0000000
  H   -2.1489398    1.2406910   -0.0000000
  H    0.0000000    2.4813820    0.0000000
'''
extra_bound = [(0,5)]
carbon_coordinates = pts.taking_carbon_coordinates(benzene_molecule)
print(carbon_coordinates)


[[ 1.2116068  0.6995215  0.       ]
 [ 1.2116068 -0.6995215  0.       ]
 [-0.        -1.399043  -0.       ]
 [-1.2116068 -0.6995215 -0.       ]
 [-1.2116068  0.6995215 -0.       ]
 [ 0.         1.399043   0.       ]]


In [88]:
mol = gto.M(atom = benzene_molecule, basis ='sto-3g' )
print(len(mol.ao_labels()))
print(mol.ao_labels())
pz_indices = [i for i, label in enumerate(mol.ao_labels()) if 'C 2pz' in label]
print(pz_indices)


36
['0 C 1s    ', '0 C 2s    ', '0 C 2px   ', '0 C 2py   ', '0 C 2pz   ', '1 C 1s    ', '1 C 2s    ', '1 C 2px   ', '1 C 2py   ', '1 C 2pz   ', '2 C 1s    ', '2 C 2s    ', '2 C 2px   ', '2 C 2py   ', '2 C 2pz   ', '3 C 1s    ', '3 C 2s    ', '3 C 2px   ', '3 C 2py   ', '3 C 2pz   ', '4 C 1s    ', '4 C 2s    ', '4 C 2px   ', '4 C 2py   ', '4 C 2pz   ', '5 C 1s    ', '5 C 2s    ', '5 C 2px   ', '5 C 2py   ', '5 C 2pz   ', '6 H 1s    ', '7 H 1s    ', '8 H 1s    ', '9 H 1s    ', '10 H 1s    ', '11 H 1s    ']
[4, 9, 14, 19, 24, 29]


In [89]:
MAXITER = 2000
E_conv = 1.0e-6

In [90]:
S = mol.intor('int1e_ovlp')
T = mol.intor('int1e_kin')
V = mol.intor('int1e_nuc')
H_core = T + V
eri = mol.intor('int2e')


In [91]:
enuc = mol.get_enuc()
ndocc = mol.nelec[0]
print(enuc, ndocc)

203.01933448917916 21


In [92]:
# ==> Inspecting S for AO orthonormality <==
def controleer_orthonormaliteit(matrix):
    is_ortho = np.allclose(np.eye(len(matrix[0])), matrix)
    if is_ortho:
        print("De matrix is orthonormaal.")
    else:
        print("De matrix is niet orthonormaal.")
controleer_orthonormaliteit(S)

De matrix is niet orthonormaal.


In [93]:
# ==> Construct AO orthogonalization matrix A <==
from scipy.linalg import fractional_matrix_power
A = fractional_matrix_power(S, -0.5)
A = np.asarray(A)
ASA = A@S@A

# Check orthonormality
controleer_orthonormaliteit(ASA)


De matrix is orthonormaal.


In [94]:
# ==> Compute C & D matrices with CORE guess <==
# Transformed Fock matrix
F_p = A @ H_core @ A.T

# Diagonalize F_p for eigenvalues & eigenvectors with NumPy
epsilon, C_p = np.linalg.eigh(F_p)

# Transform C_p back into AO basis
C = A @ C_p
# Grab occupied orbitals
C_occ = C[:, :ndocc]

# Build density matrix from occupied orbitals
D = np.einsum('ik, kj -> ij',C_occ, C_occ.T, optimize=True)
print(D)

[[ 1.05161681e+00 -2.63961502e-01  1.73851319e-01 ...  5.06437947e-04
   8.34438602e-04  2.39235416e-03]
 [-2.63961502e-01  1.62463133e+00 -1.05463266e+00 ... -3.95740403e-03
  -1.60496482e-03 -4.05175605e-02]
 [ 1.73851319e-01 -1.05463266e+00  1.22840846e+00 ...  2.26210092e-03
   6.30035157e-03  3.40628298e-02]
 ...
 [ 5.06437947e-04 -3.95740403e-03  2.26210092e-03 ...  1.24946462e-02
  -5.53348775e-04 -9.60171853e-04]
 [ 8.34438602e-04 -1.60496482e-03  6.30035157e-03 ... -5.53348775e-04
   1.24946462e-02 -5.53347750e-04]
 [ 2.39235416e-03 -4.05175605e-02  3.40628298e-02 ... -9.60171853e-04
  -5.53347750e-04  1.24946421e-02]]


In [95]:
def apply_lagrange_multipliers(F, constraints, lambdas):
    for mu in range(len(constraints)):
        C_mu = constraints[mu]
        F += lambdas[mu] * np.outer(C_mu, C_mu)
    return F

In [96]:
def normaliseer(eigenvectors):
    eigenvectors = eigenvectors.T
    for index, eigenvector in enumerate(eigenvectors):
        eigenvectors[index] = eigenvector * (np.dot(eigenvector, eigenvector))**(-1/2)
    return eigenvectors.T

In [97]:

# ==> SCF Iterations <==
# Pre-iteration energy declarations
SCF_E = 0.0
E_old = 0.0
constraints = [np.zeros(H_core.shape[0])]
constraints[0][4] = 1.5
lambdas = [0.1]
print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(1, MAXITER + 1):

    # Build Fock matrix
    J = np.einsum('pqrs,rs->pq', eri, D, optimize=True)
    K = np.einsum('prqs,rs->pq', eri, D, optimize=True)
    
    F = H_core + 2 * J - K
    #F = apply_lagrange_multipliers(F, constraints, lambdas)
    # Compute RHF energy
    SCF_E = enuc + np.einsum('pq,pq->', (H_core + F), D, optimize=True)
    SCF_E = enuc + np.sum( D * (H_core + F))
    
    
    # SCF Converged?
    if (abs(SCF_E - E_old) < E_conv):
        break
    E_old = SCF_E

    # Compute new orbital guess
    epsilon, C_p = np.linalg.eigh(A @ F @ A)
    C = A @ C_p
    C_occ = C[:, :ndocc]  
    D= np.einsum('pi,iq->pq', C_occ, C_occ.T, optimize=True)

    D_2 = np.einsum('pi,iq->pq', normaliseer(C)[:, :ndocc], normaliseer(C)[:, :ndocc].T, optimize=True)
    print(D_2[4][4]*2)

    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        raise Exception("Maximum number of SCF iterations exceeded.")
# Post iterations
print(sum(D_2.diagonal()*2))
print('\nSCF converged.')
print('Final RHF Energy: %.8f [Eh]' % (SCF_E))

==> Starting SCF Iterations <==

1.0000000206382234
1.0000000005783667
1.000000011016953
1.0000000038921084
1.0000000059866623
1.0000000047755484
42.000000000000014

SCF converged.
Final RHF Energy: -227.89028017 [Eh]
